<a href="https://colab.research.google.com/github/sergioarnold87/Practica_Sergio/blob/main/TD3_Sergio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradiente de política determinista profunda (TD3) de doble retardo

## Instalación de los paquetes

In [1]:
!pip install pybullet

     |████████████████████████████████| 89.3 MB 29 kB/s 


## Importar las librerías

In [2]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from gym import wrappers
from torch.autograd import Variable
from collections import deque

## Paso 1: Inicializar la memoria de la repeteción de experiencias

In [4]:
"""
La clase RipleyBuffers define unos conceptos, una plantilla y las futuras instancias de esta clase van a ser los objetos
que no serán ni más ni menos que la memoria en sí mismo que utilizaremos en el código. De hecho, este shelf que aparece 
por aquí y que ya he guardado en él algunas variables harán referencia al propio objeto memoria que será creado a partir de la 
clase RipleyBuffer.
"""

class ReplayBuffer(object):
  def __init__(self, max_size =1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

# -----------------------------------------------------------------------------
 # Vamos a construir la primera herramienta, la primera funcionalidad de esta 
 # clase. el metodo ADD
# -----------------------------------------------------------------------------
  def add(self, transition):
    if len(self.storage)== self.max_size:
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr +1) % self.max_size
    else:
      self.storage.append(transition)


# -----------------------------------------------------------------------------
  #CREAR EL METODO SAMPLE

  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), size = batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind:
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy = False))
      batch_next_states.append(np.array(new_state, copy = False))
      batch_actions.append(np.array(actions, copy = False))
      batch_rewards.append(np.array(reward, copy = False))
      batch_dones.append(np.array(done, copy = False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)

## Paso 2: Construimos una red neuronal para el **actor del modelo** y una red neuronal para el **actor del objetivo**

In [5]:
class Actor(nn.Module):

  def __init__(self, state_dim, action_dim_max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    señf.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.tanh(self.layer_3(x))
    return x